In [1]:
import argparse
import pandas as pd
import os

In [2]:
dirs_dct = dict(list(pd.read_csv('../directory_paths.csv')['paths'].apply(eval)))

In [3]:
nicu_concept_list=['positive_Septicemia', 'positive_Jaundice', 'positive_Other perinatal conditions', 'positive_Suspected Infection', 'positive_Tuesday', 'positive_Premature birth', 'positive_Patent ductus arteriosus', 'positive_Needs Hepatitis Vaccination', 'positive_Chronic respiratory disease', 'positive_MALE', 'positive_FEMALE', 'positive_Single Birth, Cesarean Section', 'positive_Sunday', 'positive_Saturday', 'positive_Low birth weight', 'positive_Sleep Apnea', 'positive_Wednesday', 'positive_Anemia of prematurity', 'positive_Monday', 'positive_Retrolental fibroplasia', 'positive_3+ Birthmates, Cesarian Section', 'positive_Bradycardia', 'positive_Friday', 'positive_Twin Birth, Cesarian Section', 'positive_Respiratory Distress Syndrome', 'positive_Thursday']

In [4]:
metabolic_concept_list =['positive_MED_ACE', 'positive_MED_INS', 'positive_OLD', 'positive_SMOKER', 'positive_MED_STAT', 'positive_MED_BETA', 'positive_MED_FIBR', 'positive_MALE', 'positive_FEMALE', 'positive_OBESE', 'positive_HYPERTENSION', 'positive_LOWHDL', 'positive_Monday', 'positive_Tuesday', 'positive_Wednesday', 'positive_DIABETES', 'positive_HYPERGLYCERIDEMIA', 'positive_YOUNG', 'positive_HYPERLIPIDEMIA', 'positive_Sunday', 'positive_Monday', 'positive_Thursday', 'positive_Friday', 'positive_Saturday']

In [5]:
common_header = """#!/bin/bash
#SBATCH --partition=gpu
#SBATCH --mem=32g
#SBATCH --cpus-per-task=16
#SBATCH --output=/data/moldwinar/joboutputs/job_%j.out
#SBATCH --error=/data/moldwinar/joboutputs/job_%j.error
#SBATCH --time=5-9:00:00
#SBATCH --gres=gpu:v100x:1,lscratch:500"""
common_header = common_header+'\n\npython tcav_experiment.py '

In [6]:
def generate_args(ckpt_suffix,concept_list,bn,wrapper,task, target,encoder):
    args = {}
    args['ckpt_path']=os.path.join(dirs_dct['checkpoints_dir'],ckpt_suffix)
    args['num_trials']=500
    args['bottleneck_name']=bn
    args['experiment_name']= '_'.join([args['ckpt_path'].split('/')[-2],args['ckpt_path'].split('/')[-1],str(args['num_trials']),args['bottleneck_name']])
    args['working_dir']=os.path.join(dirs_dct['working_dir'],args['experiment_name'])
    args['source_dir']=dirs_dct['source_dir']
    args['results_dir'] = os.path.join(dirs_dct['working_dir'],'results',args['experiment_name'])
    args['target'] = target
    args['activations_dir'] = 'LSCRATCH'
    args['all_activations_dir']=os.path.join(args['activations_dir'],'all_activations/')
    args['concepts_dir'] =os.path.join(dirs_dct['concepts_dir'],task)
    args['data_encode_function_name']=encoder
    args['wrapper_name']=wrapper
    args['concepts_list'] ='\"'+str(concept_list)+'\"'
    args_string = '\n'.join([' '.join(['--'+k,str(v), '\\']) for k,v in args.items()])[:-1]
#     print(args)
    bash_text =  common_header+args_string
    with open(os.path.join('bash_scripts/',args['wrapper_name'][:-7]+'_'+task+'.sh'),'w') as shfile:
        shfile.write(bash_text)

In [7]:
generate_args(ckpt_suffix='NICU_MLP_0/checkpoint-28853',
              concept_list=nicu_concept_list, bn='pool',wrapper='MLPWrapper',task='NICU', target='nicu_los',encoder='bert_encode')

In [8]:
generate_args(ckpt_suffix='CCS_MLP_TEXT__0/checkpoint-14877',
              concept_list=metabolic_concept_list, bn='pool',wrapper='MLPWrapper',task='metabolic', target='CCS',encoder='bert_encode')

In [9]:
generate_args(ckpt_suffix='CCS_BERT_TEXT__0/checkpoint-4959',
              concept_list=metabolic_concept_list, bn='pool',wrapper='BertWrapper',task='metabolic', target='CCS',encoder='bert_encode')

In [10]:
generate_args(ckpt_suffix='FINALWeightedBlueBert512NicuLosAdmissions_2/checkpoint-26015',
              concept_list=nicu_concept_list, bn='pool',wrapper='BertWrapper',task='NICU', target='nicu_los',encoder='bert_encode')

In [11]:
generate_args(ckpt_suffix='logistic/best_steps_84.pt',
              concept_list=nicu_concept_list, bn='pre_sum',wrapper='LogisticWrapper',task='NICU', target='nicu_los',encoder='logistic_bert_encode')

In [12]:
generate_args(ckpt_suffix='FINALLogisticCCS_halfsplit_0/best_steps_13.pt',
              concept_list=metabolic_concept_list, bn='pre_sum',wrapper='LogisticWrapper',task='metabolic', target='CCS',encoder='logistic_bert_encode')

In [13]:
generate_args(ckpt_suffix='CNN_NICU/best_steps_420.pt',
              concept_list=nicu_concept_list, bn='dropout',wrapper='CnnWrapper',task='NICU', target='nicu_los',encoder='bert_encode')

In [14]:
generate_args(ckpt_suffix='CNN_METABOLIC/best_steps_414.pt',
              concept_list=metabolic_concept_list, bn='dropout',wrapper='CnnWrapper',task='metabolic', target='CCS',encoder='bert_encode')

In [19]:
generate_args(ckpt_suffix='CNN_RES_METABOLIC/best_steps_1242.pt',
              concept_list=metabolic_concept_list, bn='final_dense',wrapper='CnnPlusStructuredWrapper',task='metabolic', target='CCS',encoder='bert_rhn_encode')

In [16]:
generate_args(ckpt_suffix='CCS_just_residual_0/checkpoint-133893',
              concept_list=metabolic_concept_list, bn='finaldense',wrapper='JustRHNWrapper',task='metabolic', target='CCS',encoder='bert_rhn_encode')

In [17]:
generate_args(ckpt_suffix='residual_CCS__0/checkpoint-13224/',
              concept_list=metabolic_concept_list, bn='finaldense',wrapper='BertRHNWrapper',task='metabolic', target='CCS',encoder='bert_rhn_encode')